In [1]:
%run ./setting.ipynb

In [2]:
NDX = pd.read_parquet(
    os.path.join(
    data_path,
    "NDX.parquet")
)

In [3]:
NDX.to_sql(
    "NDX", 
    conn,
    if_exists = "replace",
    )

9514

# last 10 days 

In [4]:
day = 1

In [5]:
features = [
    'Open',
    'Close',
    'High',
    'Low',
    'Volume',
    'close_to_open_ratio',
    'high_to_low_ratio',
]

features_cat = [
    'daily_rise'
]

In [6]:
sql(f"""
select 
row_number() over (order by Date asc) as Date_rank,
*, 
Close/Open as close_to_open_ratio,
High/Low as high_to_low_ratio,
case
    when 
    Close > Open
    then 'rise'
    else 'drop'
end as daily_rise
from NDX
""").to_sql(
    "raw_feature", 
    conn,
    if_exists = "replace",
    )

9514

In [7]:
days = range(1,10)

In [8]:
for day in days:
    
    sql_feature_columns = []
    
    for c in features:
        sql_feature_columns.append(f"b.{c} as feature_num__{c}__{day}_day_ago")
    for c in features_cat:
        sql_feature_columns.append(f"b.{c} as feature_cat__{c}__{day}_day_ago")

    sql_feature_columns = ",\n".join(sql_feature_columns)

    sql(f"""
    select 
    a.Date,
    {sql_feature_columns}
    from raw_feature as a
    left join raw_feature as b
    on b.Date_rank + {day} = a.Date_rank
    """).to_sql(
        f"daily_feature_{day}_day_ago", 
        conn,
        if_exists = "replace",
        )
    
    print(f"daily_feature_{day}_day_ago")

daily_feature_1_day_ago
daily_feature_2_day_ago
daily_feature_3_day_ago
daily_feature_4_day_ago
daily_feature_5_day_ago
daily_feature_6_day_ago
daily_feature_7_day_ago
daily_feature_8_day_ago
daily_feature_9_day_ago


# feature merging

## base table

In [9]:
lable = pd.read_parquet(
os.path.join(
data_path,
"NDX_daily_label.parquet")
)

lable.to_sql(
    "lable", 
    conn,
    if_exists = "replace",
    )

9514

In [10]:
sql(f"""
select distinct Date
from lable
order by Date asc
""").to_sql(
    "stage_000", 
    conn,
    if_exists = "replace",
    )

9514

In [11]:
sql(f"""
select * 
from stage_000
limit 10
""")

,index,Date
0,0,1985-10-01 00:00:00-04:00
1,1,1985-10-02 00:00:00-04:00
2,2,1985-10-03 00:00:00-04:00
3,3,1985-10-04 00:00:00-04:00
4,4,1985-10-07 00:00:00-04:00
5,5,1985-10-08 00:00:00-04:00
6,6,1985-10-09 00:00:00-04:00
7,7,1985-10-10 00:00:00-04:00
8,8,1985-10-11 00:00:00-04:00
9,9,1985-10-14 00:00:00-04:00


In [12]:
feature_tables = {
"daily_feature_1_day_ago":["Date"],
"daily_feature_2_day_ago":["Date"],
"daily_feature_3_day_ago":["Date"],
"daily_feature_4_day_ago":["Date"],
"daily_feature_5_day_ago":["Date"],
"daily_feature_6_day_ago":["Date"],
"daily_feature_7_day_ago":["Date"],
"daily_feature_8_day_ago":["Date"],
"daily_feature_9_day_ago":["Date"],
}

In [13]:
index = 0

for table in feature_tables:
    
    index += 1

    columns = sql(f"""
    select * from {table} limit 1
    """)
    
    columns_last = sql(f"""
    select * from stage_{index-1:03d} limit 1
    """)

    ## features
    
    sql_feature = []

    for c in columns:
        if 'feature_' in c:
            sql_feature.append(f"b.{c}")
    for c in columns_last:
        if 'feature_' in c:
            sql_feature.append(f"a.{c}")

    sql_feature = ",\n".join(sql_feature)

    ## joins 
    
    sql_join = []

    for c in feature_tables[table]:
        sql_join.append(f"a.{c} = b.{c}")

    sql_join = "\nand ".join(sql_join)
    
    ## excute sql

    sql(f"""
    select 
    a.Date,
    {sql_feature}
    from stage_{index-1:03d} as a
    left join {table} as b
    on {sql_join}
    """).to_sql(
        f"stage_{index:03d}", 
        conn,
        if_exists = "replace",
        )
    
    print(f"stage_{index:03d}")

stage_001
stage_002
stage_003
stage_004
stage_005
stage_006
stage_007
stage_008
stage_009


In [14]:
sql(f"""
select 
count(*),
count(distinct Date)
from stage_{index:03d}
""")

,count(*),count(distinct Date)
0,9514,9514


In [15]:
sql(f"""
select *
from stage_{index:03d}
order by Date asc
""").to_sql(
    "features", 
    conn,
    if_exists = "replace",
    )

9514

# save to parquet

In [16]:
sql(f"""
select *
from stage_{index:03d}
order by Date asc
""").to_parquet(
    os.path.join(
    data_path,
    "NDX_features.parquet"),
    index = False
)

# end